<a href="https://colab.research.google.com/github/Mular56/Machine_learning/blob/main/%D0%9C%D1%83%D0%BB%D1%8F%D1%80_%D0%BF%D1%80_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357276 sha256=47c1c95486b1e284e83dac175b33f2d101cbe007ea856dd76f72f817eb7edb02
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
'''Завдання 1.
1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.'''
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

data = Dataset.load_builtin("ml-100k")

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)


RMSE: 0.9385


0.9385021623249028

In [6]:
'''2. Виведіть перші 5 рядків завантаженого датасету.'''
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [7]:
'''3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.'''
'''4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.'''
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [25, 100],
    'lr_all': [0.005, 0.015],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 3],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
grid_search_knn.fit(data)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [8]:
'''5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).'''

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")


Найкращий MAE для SVD: 0.7277406394332578
Найкращий MAE для KNNBasic: 0.8090021903244998
Вибраний алгоритм: SVD


In [9]:
'''6. Виведи рекомендації (10 фільмів) для конкретного користувача.'''

best_model.fit(trainset)

# Виведення рекомендацій для конкретного користувача
user_id = str(114)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання рекомендацій
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 114: 38
Топ-10 фільмів, рекомендованих для користувача:
Фільм 408 з прогнозованим рейтингом 4.57
Фільм 641 з прогнозованим рейтингом 4.30
Фільм 1019 з прогнозованим рейтингом 4.22
Фільм 657 з прогнозованим рейтингом 4.14
Фільм 613 з прогнозованим рейтингом 4.11
Фільм 709 з прогнозованим рейтингом 4.11
Фільм 114 з прогнозованим рейтингом 4.10
Фільм 745 з прогнозованим рейтингом 4.09
Фільм 1449 з прогнозованим рейтингом 4.09
Фільм 853 з прогнозованим рейтингом 4.07


In [11]:
'''Завдання 2.
1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.'''
from surprise import NMF, NormalPredictor
from surprise.model_selection import cross_validate

# Алгоритм 1: NMF (Non-negative Matrix Factorization)
algo_nmf = NMF()

# Алгоритм 2: Random (випадковий прогноз)
algo_random = NormalPredictor()

# Крос-валідація для NMF
cv_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Крос-валідація для випадкового прогнозу
cv_random = cross_validate(algo_random, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"Результати для NMF:\nRMSE: {cv_nmf['test_rmse'].mean()}\nMAE: {cv_nmf['test_mae'].mean()}")
print(f"Результати для випадкового прогнозу:\nRMSE: {cv_random['test_rmse'].mean()}\nMAE: {cv_random['test_mae'].mean()}")


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9639  0.9550  0.9583  0.9721  0.9657  0.9630  0.0059  
MAE (testset)     0.7580  0.7512  0.7549  0.7643  0.7586  0.7574  0.0044  
Fit time          3.03    2.16    2.90    2.12    4.11    2.86    0.73    
Test time         0.10    0.25    0.10    0.25    0.35    0.21    0.10    
Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5304  1.5204  1.5268  1.4991  1.5191  1.5192  0.0108  
MAE (testset)     1.2260  1.2201  1.2268  1.2040  1.2207  1.2195  0.0082  
Fit time          0.11    0.13    0.14    0.14    0.13    0.13    0.01    
Test time         0.09    0.21    0.09    0.21    0.09    0.14    0.06    
Результати для NMF:
RMSE: 0.9630103810391095
MAE: 0.7573839620388767
Результати для випадкового прогнозу:
RMSE: 1.5191565612995306

In [12]:
'''2. Отримайте рекомендацію для певного користувача.'''
# Тренуємо модель NMF на тренувальному наборі
algo_nmf.fit(trainset)

# Рекомендації для користувача (наприклад, з ID 114)
user_id = str(114)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]

# Прогнозуємо для фільмів, які користувач ще не бачив
unrated_items = all_items - rated_items
predictions = [(item, algo_nmf.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих рекомендацій
print(f"Топ-10 рекомендацій для користувача {user_id}:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")


Топ-10 рекомендацій для користувача 114:
Фільм 1367 з прогнозованим рейтингом 4.63
Фільм 1536 з прогнозованим рейтингом 4.43
Фільм 718 з прогнозованим рейтингом 4.40
Фільм 853 з прогнозованим рейтингом 4.39
Фільм 1558 з прогнозованим рейтингом 4.37
Фільм 1167 з прогнозованим рейтингом 4.36
Фільм 854 з прогнозованим рейтингом 4.35
Фільм 316 з прогнозованим рейтингом 4.31
Фільм 1642 з прогнозованим рейтингом 4.28
Фільм 524 з прогнозованим рейтингом 4.27


In [ ]:
'''
Висновки:
У ході виконання лабораторної роботи було розглянуто тему рекомендаційних
систем на прикладі використання бібліотеки Surprise. Ми завантажили датасет
для рецензій (ml-100k) та реалізували два алгоритми для рекомендаційної системи:
SVD та KNNBasic. Для обох алгоритмів було проведено крос-валідацію з метою
підбору оптимальних параметрів, що дозволило покращити точність прогнозів. На
основі метрики MAE було вибрано найкращий алгоритм, і на його основі було
виведено рекомендації для конкретного користувача.

У другій частині завдання було побудовано власну рекомендаційну систему за
допомогою алгоритму NMF (Non-negative Matrix Factorization), а також проведено
її оцінку. Ми порівняли результати алгоритму NMF із випадковим прогнозуванням,
що показало перевагу використання методів факторизації матриць для побудови
ефективних рекомендаційних систем. Загалом, робота дозволила ознайомитися з
основними підходами до побудови рекомендаційних систем, принципами налаштування
моделей та їх оцінювання на основі різних метрик.
'''